In [1]:
import pandas as pd
data = pd.read_csv('../data/raw_postpro.csv', encoding = 'cp949')
# 컬럼 삭제
df = data.drop(['Part No.2', '미입고 기간', '선박입고','완료 여부'], axis=1)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
y = df['key2']
le = preprocessing.LabelEncoder()
y = y.values.ravel()  # y를 1차원 배열로 변환
y = le.fit_transform(y)

In [3]:
from sklearn import preprocessing
label_encoders = {}  # 각 열에 대한 LabelEncoder를 저장하기 위한 딕셔너리
columns_to_encode = ["Subject",'Machinery', 'Assembly' , "Part No.1","청구품목", 'key2']  # 인코딩을 수행할 열의 이름 리스트

for column in columns_to_encode:
    le = preprocessing.LabelEncoder()
    le.fit(df[column])
    label_encoders[column] = le # 딕셔너리에 저장
    df[column+"_encoded"] = le.transform(df[column]) # 새로운 encoding 된 컬럼 추가

In [4]:
# 인코딩된 컬럼을 사용
encoded_columns = ['Subject_encoded','Machinery_encoded', 'Assembly_encoded', 'Part No.1_encoded', '청구품목_encoded']
X = df[encoded_columns].values
y = df['key2_encoded'].values

In [5]:
X,y

(array([[ 940,  108,  281, 5759, 4848],
        [1647,  113, 1180, 5759, 3692],
        [1648,  113,  206, 5759, 5915],
        ...,
        [ 568,    0,  900, 8123, 4151],
        [ 233,    4, 1210, 8124, 1592],
        [ 233,    4, 1210, 8125, 4718]]),
 array([18, 18, 18, ...,  5, 20, 20]))

In [6]:
# 데이터를 학습용과 검증용으로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print(X_train.shape)
print(X_train.dtype)


X_train shape: (16413, 5)
y_train shape: (16413,)
(16413, 5)
int32


In [8]:
import numpy as np
from keras.layers import Input, Embedding, Dense, concatenate, Flatten

embedding_input_dim_1 = int(np.max(X_train[:, 0])) + 2
embedding_input_dim_2 = int(np.max(X_train[:, 1])) + 2
embedding_input_dim_3 = int(np.max(X_train[:, 2])) + 2 # Xtrain 길이가 Xtest 보다 더 길어야 하므로 읨의 조정
embedding_input_dim_4 = int(np.max(X_train[:, 3])) + 2
embedding_input_dim_5 = int(np.max(X_train[:, 4])) + 2

# 임베딩 설정
embedding_dim = 8

# 입력 정의
input1 = Input(shape=(1,), dtype="int32", name="Subject_Input")
input2 = Input(shape=(1,), dtype="int32", name="Machinery_Input")
input3 = Input(shape=(1,), dtype="int32", name="Assembly_Input")
input4 = Input(shape=(1,), dtype="int32", name="Part_No.1_Input")
input5 = Input(shape=(1,), dtype="int32", name="청구품목_Input")


# input1 = Input(shape=(1,), name="Assembly_Input")
# input2 = Input(shape=(1,), name="Resource_Input")
# input3 = Input(shape=(1,), name="Method_Input")
# input4 = Input(shape=(1,), name="Dwg_Input")


embedding1 = Embedding(embedding_input_dim_1, embedding_dim, name="Subject_Embedding")(input1)
embedding2 = Embedding(embedding_input_dim_2, embedding_dim, name="Assembly_Embedding")(input2)
embedding3 = Embedding(embedding_input_dim_3, embedding_dim, name="Resource_Embedding")(input3)
embedding4 = Embedding(embedding_input_dim_4, embedding_dim, name="Method_Embedding")(input4)
embedding5 = Embedding(embedding_input_dim_5, embedding_dim, name="Dwg_Embedding")(input5)

# model1

In [9]:
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, concatenate, Flatten

# # 출력 통합
# merged_output = concatenate([embedding1, embedding2, embedding3, embedding4, embedding5], axis=-1)

# # Dense 계층
# dense_output = Dense(1, activation='sigmoid', name='Dense_output')(merged_output)
# output = Flatten()(dense_output)

# prediction = Dense(61, activation='softmax', name="Prediction_output")(output)
# # 모델 생성
# model1 = Model(inputs=[input1, input2, input3, input4, input5], outputs=prediction)

In [10]:
from tensorflow.keras.optimizers import Adam
# # 모델 컴파일
# learning_rate = 0.001  # 원하는 학습률 값 설정하기
# optimizer = Adam(learning_rate=learning_rate)
# # 모델 컴파일
# model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model1.summary()

In [11]:
# from tensorflow.keras.utils import to_categorical
# y_train_categorical = to_categorical(y_train, num_classes=61)
# y_test_categorical = to_categorical(y_test, num_classes=61)

In [12]:
# # 모델 훈련 및 콜백 설정
# model1.fit([X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3],  X_train[:, 4]], y_train_categorical, epochs=50, batch_size=64)

In [13]:
# y_pred1 = model1.predict([X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3], X_test[:, 4]])

In [14]:
# y_pred_classes1 = np.argmax(y_pred1, axis=1)
# # 정답과 비교할 때 카테고리화 된 y_test 필요
# y_test_classes1 = np.argmax(y_test_categorical, axis=1)

In [15]:
# y_pred_classes1,y_test_classes1

In [16]:
# # 정확도 계산
# correct = np.sum(y_pred_classes1 == y_test_classes1)
# accuracy = correct / y_test_classes1.shape[0]

# print("Accuracy: {:.2f}%".format(accuracy * 100))

In [17]:
# from sklearn.metrics import f1_score
# f1 = f1_score(y_test_classes1, y_pred_classes1, average='weighted')
# print("F1 Score: {:.2f}%".format(f1 * 100))


# model2 : model1 + lstm

In [18]:
from keras.layers import LSTM, Dense,Input, Embedding, concatenate, Flatten
from keras.models import Sequential, Model

merged_output = concatenate([embedding1, embedding2, embedding3, embedding4,embedding5], axis=-1)
lstm_output = LSTM(128)(merged_output)

dense_output = Dense(64, activation='relu', name='Dense_output')(lstm_output)
prediction = Dense(61, activation='softmax', name="Prediction_output")(dense_output)

model2 = Model(inputs=[input1, input2, input3, input4, input5], outputs=prediction)

In [19]:
from tensorflow.keras.optimizers import Adam
# 모델 컴파일
learning_rate = 0.001  # 원하는 학습률 값 설정하기
optimizer = Adam(learning_rate=learning_rate)
# 모델 컴파일
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Subject_Input (InputLayer)     [(None, 1)]          0           []                               
                                                                                                  
 Machinery_Input (InputLayer)   [(None, 1)]          0           []                               
                                                                                                  
 Assembly_Input (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 Part_No.1_Input (InputLayer)   [(None, 1)]          0           []                               
                                                                                              

In [20]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, num_classes=61)
y_test_categorical = to_categorical(y_test, num_classes=61)

In [21]:
# 모델 훈련 및 콜백 설정
model2.fit([X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3], X_train[:, 4]], y_train_categorical, epochs=300, batch_size=64)

Epoch 1/300
257/257 [==============================] - 3s 2ms/step - loss: 3.2734 - accuracy: 0.1975
Epoch 2/300
257/257 [==============================] - 1s 2ms/step - loss: 2.4324 - accuracy: 0.2987
Epoch 3/300
257/257 [==============================] - 1s 2ms/step - loss: 1.6592 - accuracy: 0.5184
Epoch 4/300
257/257 [==============================] - 1s 3ms/step - loss: 1.0723 - accuracy: 0.6962
Epoch 5/300
257/257 [==============================] - 1s 2ms/step - loss: 0.6590 - accuracy: 0.8192
Epoch 6/300
257/257 [==============================] - 1s 2ms/step - loss: 0.4031 - accuracy: 0.8902
Epoch 7/300
257/257 [==============================] - 1s 2ms/step - loss: 0.2416 - accuracy: 0.9326
Epoch 8/300
257/257 [==============================] - 1s 2ms/step - loss: 0.1449 - accuracy: 0.9620
Epoch 9/300
257/257 [==============================] - 1s 2ms/step - loss: 0.0881 - accuracy: 0.9788
Epoch 10/300
257/257 [==============================] - 1s 2ms/step - loss: 0.0542 - accura

In [22]:
y_pred2 = model2.predict([X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3], X_test[:, 4]])

129/129 [==============================] - 0s 849us/step


In [23]:
y_pred_classes2 = np.argmax(y_pred2, axis=1)
# 정답과 비교할 때 카테고리화 된 y_test 필요
y_test_classes2 = np.argmax(y_test_categorical, axis=1)

In [24]:
y_pred_classes2,y_test_classes2

(array([54, 33, 24, ..., 20, 25, 40], dtype=int64),
 array([54, 33, 24, ..., 51, 25, 40], dtype=int64))

In [26]:
# 정확도 계산
correct = np.sum(y_pred_classes2 == y_test_classes2)
accuracy = correct / y_test_classes2.shape[0]

print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 74.27%


In [27]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test_classes2, y_pred_classes2, average='weighted')
print("F1 Score: {:.2f}%".format(f1 * 100))

F1 Score: 77.90%


# model3 : model1 + lstm + cnn

In [28]:
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, concatenate, Flatten, Conv1D, GlobalMaxPooling1D
# 출력 통합
merged_output = concatenate([embedding1, embedding2, embedding3, embedding4, embedding5], axis=-1)


# CNN Layer
cnn_output = Conv1D(filters=128, kernel_size=1, activation='relu')(merged_output)  # kernel_size 수정
cnn_output = GlobalMaxPooling1D()(cnn_output)

# LSTM Layer
lstm_output = LSTM(128)(merged_output)

# Dense Layer
dense_input = concatenate([cnn_output, lstm_output], axis=-1)
dense_output = Dense(64, activation='relu', name='Dense_output')(dense_input)

prediction = Dense(61, activation='softmax', name="Prediction_output")(dense_output)

# 모델 생성
model3 = Model(inputs=[input1, input2, input3, input4, input5], outputs=prediction)

In [29]:
from tensorflow.keras.optimizers import Adam
# 모델 컴파일
learning_rate = 0.001  # 원하는 학습률 값 설정하기
optimizer = Adam(learning_rate=learning_rate)
# 모델 컴파일
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model3.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Subject_Input (InputLayer)     [(None, 1)]          0           []                               
                                                                                                  
 Machinery_Input (InputLayer)   [(None, 1)]          0           []                               
                                                                                                  
 Assembly_Input (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 Part_No.1_Input (InputLayer)   [(None, 1)]          0           []                               
                                                                                            

In [30]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train, num_classes=61)
y_test_categorical = to_categorical(y_test, num_classes=61)

In [31]:
# 모델 훈련 및 콜백 설정
model3.fit([X_train[:, 0], X_train[:, 1], X_train[:, 2], X_train[:, 3], X_train[:, 4]], y_train_categorical, epochs=300, batch_size=64)

Epoch 1/300
257/257 [==============================] - 3s 2ms/step - loss: 1.8310 - accuracy: 0.5716
Epoch 2/300
257/257 [==============================] - 1s 2ms/step - loss: 0.1930 - accuracy: 0.9739
Epoch 3/300
257/257 [==============================] - 1s 2ms/step - loss: 0.0296 - accuracy: 0.9978
Epoch 4/300
257/257 [==============================] - 1s 3ms/step - loss: 0.0096 - accuracy: 0.9995
Epoch 5/300
257/257 [==============================] - 1s 2ms/step - loss: 0.0052 - accuracy: 0.9997
Epoch 6/300
257/257 [==============================] - 1s 3ms/step - loss: 0.0031 - accuracy: 0.9998
Epoch 7/300
257/257 [==============================] - 1s 3ms/step - loss: 0.0020 - accuracy: 0.9999
Epoch 8/300
257/257 [==============================] - 1s 2ms/step - loss: 0.0016 - accuracy: 0.9999
Epoch 9/300
257/257 [==============================] - 1s 3ms/step - loss: 9.9092e-04 - accuracy: 0.9999
Epoch 10/300
257/257 [==============================] - 1s 3ms/step - loss: 7.4322e-04 

In [32]:
y_pred3 = model3.predict([X_test[:, 0], X_test[:, 1], X_test[:, 2], X_test[:, 3], X_test[:, 4]])

129/129 [==============================] - 1s 841us/step


In [33]:
y_pred_classes3 = np.argmax(y_pred3, axis=1)
# 정답과 비교할 때 카테고리화 된 y_test 필요
y_test_classes3 = np.argmax(y_test_categorical, axis=1)

In [34]:
y_pred_classes3,y_test_classes3

(array([54, 33, 24, ..., 18, 25, 40], dtype=int64),
 array([54, 33, 24, ..., 51, 25, 40], dtype=int64))

In [36]:
# 정확도 계산
correct = np.sum(y_pred_classes3 == y_test_classes3)
accuracy = correct / y_test_classes3.shape[0]

print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 77.00%


In [37]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test_classes3, y_pred_classes3, average='weighted')
print("F1 Score: {:.2f}%".format(f1 * 100))

F1 Score: 80.54%
